In [15]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from geopy.distance import geodesic
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from math import radians

In [2]:
df = pd.read_csv('hotels.csv', encoding='ISO-8859-1')

# 1. Data Cleaning & Preprocessing

## 1.1 Basic Data Understanding

In [3]:
df.columns

Index(['countyCode', ' countyName', ' cityCode', ' cityName', ' HotelCode',
       ' HotelName', ' HotelRating', ' Address', ' Attractions',
       ' Description', ' FaxNumber', ' HotelFacilities', ' Map',
       ' PhoneNumber', ' PinCode', ' HotelWebsiteUrl'],
      dtype='object')

In [4]:
df.shape

(1010033, 16)

## 1.2 Geospatial Data Cleaning

In [5]:
# remove leading/trailing spaces from all column names
df.rename(columns=lambda x: x.strip(), inplace=True)  

# split the 'Map' column (which contains "latitude|longitude" strings)
df[['latitude', 'longitude']] = df['Map'].str.split('|', expand=True)

# convert latitude and longtitude column from string to float 
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# drop the original 'Map' column
df.drop(columns=['Map'], inplace=True)

# drop rows with missing or invalid coordinates
df = df[df['latitude'].notna() & df['longitude'].notna()]


In [6]:
df.shape

(1009101, 17)

## 1.3 Handle Missing Value

In [7]:
df.isnull().sum()

countyCode            912
countyName              0
cityCode                0
cityName                0
HotelCode               0
HotelName               0
HotelRating             0
Address               101
Attractions        524475
Description         46871
FaxNumber          559594
HotelFacilities     49502
PhoneNumber        326461
PinCode             30383
HotelWebsiteUrl    249240
latitude                0
longitude               0
dtype: int64

In [8]:
# Drop unneeded columns
df.drop(columns=['FaxNumber', 'PinCode'], inplace=True)

# Fill missing text fields
text_cols = ['Address', 'Attractions', 'Description', 'HotelFacilities']
df[text_cols] = df[text_cols].fillna('')

# Handle countyCode
df['countyCode'] = df['countyCode'].fillna('unknown')


In [9]:
# Filter hotels that have zero useful content (i.e. all 4 text fields empty)
df['info_score'] = df[text_cols].apply(lambda x: sum([bool(s.strip()) for s in x]), axis=1)
df = df[df['info_score'] >= 1]  # keep hotels with at least 1 non-empty text feature

In [10]:
df.shape

(1009007, 16)

In [11]:
rating_order = {
    'OneStar': 1,
    'TwoStar': 2,
    'ThreeStar': 3,
    'FourStar': 4,
    'FiveStar': 5,
    'All': 0  # optional catch-all
}

# Create a numerical column for easier sorting
df['RatingValue'] = df['HotelRating'].map(rating_order)


# 1.4 Clusterring to remove locations outlier

In [16]:
# Prepare lat-lon array
coords = df[['latitude', 'longitude']].to_numpy()

# Convert degrees to radians
coords_rad = np.radians(coords)

# DBSCAN parameters
kms_per_radian = 6371.0088
epsilon = 5 / kms_per_radian  # 5km radius

# Cluster
db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(coords_rad)
df['Cluster'] = db.labels_

# Remove outliers (label = -1)
df = df[df['Cluster'] != -1]
df.drop(columns = ['Cluster'], inplace = True)

C:\Users\Ong Hui Ling\AppData\Local\Temp\ipykernel_27240\3952928081.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ['Cluster'], inplace = True)


# 2. Recommendation System Modelling

# 2.1 Location-Based Filtering (with optional radius)

In [17]:
from geopy.distance import geodesic

# Function to filter by city and find nearby hotels (based on latitude/longitude)
def filter_by_location(city_name, radius_km=10):
    # Filter by city
    city_hotels = df[df['cityName'] == city_name]
    
    # Get the base city hotel's coordinates
    base_lat = city_hotels.iloc[0]['latitude']
    base_lon = city_hotels.iloc[0]['longitude']
    
    base_coord = (base_lat, base_lon)
    
    # Calculate distances for all hotels in the city
    city_hotels['distance_km'] = city_hotels.apply(
        lambda row: geodesic(base_coord, (row['latitude'], row['longitude'])).km, axis=1)
    
    # Filter by radius 
    nearby_hotels = city_hotels[city_hotels['distance_km'] <= radius_km]
    
    return nearby_hotels


# 2.2 Content-Based Filtering (TF-IDF + NearestNeighbors)

In [18]:
# Create a combined text feature for content-based filtering
df['combined_features'] = df['HotelFacilities'] + ' ' + df['Attractions'] + ' ' + df['Description']

C:\Users\Ong Hui Ling\AppData\Local\Temp\ipykernel_27240\3200295879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_features'] = df['HotelFacilities'] + ' ' + df['Attractions'] + ' ' + df['Description']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Vectorize the combined text features
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

In [ ]:
# Fit the NearestNeighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

# 2.3 Recommend Similar Hotel (Locations, Ratings, Facilities)

In [ ]:
def recommend_similar_hotels_priority(hotel_name, city_name, radius_km=10, top_n=5):
    # Get selected hotel info
    selected_hotel = df[df['HotelName'] == hotel_name].iloc[0]
    selected_rating = selected_hotel['HotelRating']
    selected_rating_value = rating_order[selected_rating]
    
    # Step 1: Filter by city (or radius if using lat/lon)
    city_hotels = df[df['cityName'].str.lower() == city_name.lower()]
    
    # Step 2: Prioritize same rating hotels
    same_rating_hotels = city_hotels[city_hotels['HotelRating'] == selected_rating]

    # Step 3: Fallback to nearby hotels with similar but lower/higher ratings (optional)
    if len(same_rating_hotels) < top_n:
        other_hotels = city_hotels[city_hotels['HotelRating'] != selected_rating]
        other_hotels = other_hotels.sort_values(by='RatingValue', ascending=False)
        fallback_hotels = pd.concat([same_rating_hotels, other_hotels]).head(top_n + 1)
    else:
        fallback_hotels = same_rating_hotels

    # Step 4: Compute TF-IDF similarity within these hotels
    tfidf_features = vectorizer.transform(fallback_hotels['combined_features'])
    selected_vector = vectorizer.transform([selected_hotel['combined_features']])
    cosine_sim = cosine_similarity(selected_vector, tfidf_features).flatten()

    fallback_hotels = fallback_hotels.copy()
    fallback_hotels['Similarity'] = cosine_sim
    fallback_hotels = fallback_hotels[fallback_hotels['HotelName'] != hotel_name]


    # Step 5: Sort by rating and similarity
    results = fallback_hotels.sort_values(by=['RatingValue', 'Similarity'], ascending=[False, False])

    return results.head(top_n)


In [ ]:
import joblib

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# Save the DataFrame
df.to_pickle('hotel_data.pkl')


In [ ]:
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')

['tfidf_matrix.pkl']

# 3. Filter to South East Asia data only to deploy to streamlit cloud

In [ ]:
south_eastasia_countries = ['Cambodia', 'Indonesia', 'Laos', 'Malaysia',
    'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam']

df = pd.read_pickle("hotel_data.pkl")  # your original large file
df_asia = df[df['Country'].isin(south_eastasia_countries)].copy()
df_asia.to_pickle("hotel_data_southeastasia.pkl", compression="gzip")  # MUCH smaller file!